In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub

ModuleNotFoundError: No module named 'tensorflow_hub'

In [2]:
# Load all the JSONs from a folder as a list of dictionaries

import json
import os

def load_jsons_from_folder(folder_path):
    jsons = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            with open(os.path.join(folder_path, filename)) as f:
                jsons.append(json.load(f))
    return jsons

labels = load_jsons_from_folder('./TMA_WSI_Labels_updated')


In [3]:
converted_data = [[(d['x'], d['y']) for d in sublist] for sublist in labels]
converted_data

[[(112.12206572769952, 957.6083333333332),
  (231.72209026128266, 946.5364777740075),
  (524.1839145638409, 941.7683583782251),
  (407.9957514604355, 944.9845990440785),
  (341.9903479236812, 943.353086419753),
  (285.78574273197927, 942.5069693349263),
  (652.8587737427418, 935.5928550339534),
  (596.9017162349429, 938.3401556337277),
  (774.1241432570711, 930.4546770923835),
  (714.5962243797194, 929.7140237324702),
  (832.6986994219653, 921.3100915221579),
  (891.4434218289085, 921.1294395280236),
  (102.80659567159051, 895.6105576548722),
  (333.790202342918, 885.879392971246),
  (281.8509147528221, 883.5565070714933),
  (463.95188101487315, 881.6034801205404),
  (524.8287393162393, 876.3053418803419),
  (714.0371822285579, 866.8131385127108),
  (650.2929755006608, 871.0219579139982),
  (596.1126442346485, 877.522149600581),
  (778.2677902621723, 863.2086974615064),
  (337.80641857289174, 832.5735745988392),
  (276.92488262910797, 830.1700871898055),
  (95.16507720254313, 836.33912

In [4]:
# Get the paths of all the images in the folder as a list

import glob
image_paths = glob.glob('./TMA WSI Padded PNGs/*.png')
image_paths.sort()
image_paths


[]

In [9]:
# Load and Preprocess the Dataset
# For the purpose of this example, we'll create a small sample dataset
# In practice, you should load your actual dataset here

# Sample dataset: list of dictionaries containing image file paths and core coordinates
sample_dataset = [
    {'image_path': image_paths[i], 'cores': converted_data[i]} for i in range(len(image_paths))
]

# Visualize the sample data
for data in sample_dataset:
    print(f"Image Path: {data['image_path']}")
    print(f"Cores Coordinates: {data['cores']}")

Image Path: ./TMA WSI Padded PNGs/158867.png
Cores Coordinates: [(112.26139005113899, 957.5426545792653), (231.71985575839528, 946.6165201712869), (524.1983947119925, 941.7881964117091), (407.9957514604355, 944.9845990440785), (341.9564145658263, 943.3389355742296), (285.7270924236414, 942.4703160121644), (652.8819894055326, 935.6046694133804), (596.9497135582432, 938.4073838319541), (774.0922269909394, 930.2824988078206), (714.5975898429093, 929.6993759414676), (832.6654650324441, 921.3108627733717), (891.4752847246682, 921.1347892450393), (102.79473144030106, 895.588664613981), (333.790202342918, 885.879392971246), (281.8533644496305, 883.5718916115649), (463.9301246738185, 881.6201797622499), (525.0581529128752, 876.0226963706724), (713.9258602341256, 866.8452169800165), (650.2382978723405, 871.0791286727456), (596.1080470891791, 877.5020964360587), (778.2677902621723, 863.2086974615064), (337.9227903826617, 832.613839033751), (276.95633652822147, 830.0567092651758), (95.15991833957

In [14]:
import torch

In [25]:
import tensorflow as tf

class TissueMicroarrayDataset:
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = tf.io.read_file(item['image_path'])
        image = tf.image.decode_jpeg(image, channels=3)  # Adjust channels as needed

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        cores = tf.convert_to_tensor(item['cores'])

        return image, cores

    def to_tf_dataset(self):
        # This method converts to a tf.data.Dataset
        def gen():
            # Generator function to supply data
            for i in range(len(self)):
                yield self.__getitem__(i)

        # Define the output signature for the dataset
        output_signature = (
            tf.TensorSpec(shape=(None, None, 3), dtype=tf.uint8),
            tf.TensorSpec(shape=(None,), dtype=tf.float32)
        )

        # Create the tf.data.Dataset from the generator
        return tf.data.Dataset.from_generator(
            gen,
            output_signature=output_signature
        )

# Assuming sample_dataset is a list of dictionaries with 'image_path' and 'cores'
tma_dataset = TissueMicroarrayDataset(sample_dataset)

# Convert to tf.data.Dataset
tf_dataset = tma_dataset.to_tf_dataset()

# Prepare dataset for training (e.g., batching, prefetching)
tf_dataset = tf_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Now tf_dataset is ready to be used with TensorFlow's model.fit


In [26]:
tf_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, None), dtype=tf.float32, name=None))>

In [31]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import UpSampling2D

# Define the input shape
input_shape = (512, 512, 3)
inputs = Input(shape=input_shape)

# Load the base model
base_model = hub.KerasLayer("https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/1", trainable=True)

# Use the base model on the inputs
cls_outputs, box_outputs = base_model(inputs)

# Concatenate the outputs
resized_outputs = [UpSampling2D(size=(64 // tensor.shape[1], 64 // tensor.shape[2]))(tensor) for tensor in box_outputs]
concatenated_outputs = Concatenate(axis=-1)(resized_outputs)

# Add additional layers
x = Dense(128, activation='relu')(concatenated_outputs)
x = Dropout(0.2)(x)

# Add the final output layer
outputs = Dense(2, activation='linear')(x)  # Output layer for X, Y coordinates

# Create the Model
model = Model(inputs=inputs, outputs=outputs)


In [32]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


In [33]:
box_outputs

[<KerasTensor: shape=(None, 64, 64, 224) dtype=float32 (created by layer 'keras_layer_2')>,
 <KerasTensor: shape=(None, 32, 32, 224) dtype=float32 (created by layer 'keras_layer_2')>,
 <KerasTensor: shape=(None, 16, 16, 224) dtype=float32 (created by layer 'keras_layer_2')>,
 <KerasTensor: shape=(None, 8, 8, 224) dtype=float32 (created by layer 'keras_layer_2')>,
 <KerasTensor: shape=(None, 4, 4, 224) dtype=float32 (created by layer 'keras_layer_2')>]

In [34]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 keras_layer_2 (KerasLayer)  ([(None, 64, 64, 224),       1509201   ['input_3[0][0]']             
                              (None, 32, 32, 224),        6                                       
                              (None, 16, 16, 224),                                                
                              (None, 8, 8, 224),                                                  
                              (None, 4, 4, 224)],                                                 
                              [(None, 64, 64, 224),                                         

In [35]:
history = model.fit(tf_dataset,
                    epochs=10,  # number of epochs to train
                    steps_per_epoch=32)  # validation steps if using a validation dataset

Epoch 1/10


2023-11-06 10:00:37.322051: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (144, 2) where an element of shape (None,) was expected.
Traceback (most recent call last):

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (144, 2) where an element of shape (None,) was expected.


2023-11-06 10:00:37.497849: W tensorflow/core/frame

InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element of shape (144, 2) where an element of shape (None,) was expected.
Traceback (most recent call last):

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (144, 2) where an element of shape (None,) was expected.


	 [[{{node 